### Dashboard - Comparar precios de supermercados chilenos

Utilizando Python y diferentes librerias se obtendrán los datos de los diferentes supermercados Jumbo, Santa Isabel, Lider y Unimarc. Estos datos luego serán cargados en Github y en Power BI con el fín de crear un Dashboard que muestre los productos de estos supermercados, con sus respectivos precios e información.

#### Definición de los scrapers

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from random import randrange
from datetime import datetime
import undetected_chromedriver as uc
import urllib.parse
import time
import re

# Función para transformar texto a formato url
def transformar(texto):
    return urllib.parse.quote(texto)

# Función para obtener los datos del supermercado Jumbo
def scrap_jumbo(busqueda,tabla):
    
    # Configuración de las opciones, estas dependen de la página
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    
    # Fecha actual
    fecha = datetime.now().date()
    
    # Tiempo de espera aleatorio entre 5 y 10 para evitar saturar la página
    tiempo = randrange(5,10)

    driver = webdriver.Chrome(options=options)
    textobusqueda = transformar(busqueda)
    url = f'https://www.jumbo.cl/busqueda?ft={textobusqueda}'
    
    driver.get(url)
    time.sleep(tiempo)
    
    # Obtiene los datos de la página
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    productos = soup.find_all('div', class_='product-card')
    
    for producto in productos:
    
        marca = producto.find('a', class_='product-card-brand')
        marca_text = marca.get_text(strip=True).title() if marca else 'Sin marca'

        nombre = producto.find('a', class_='product-card-name')
        nombre_text = nombre.get_text(strip=True).title() if nombre else 'Sin nombre'

        precio = producto.find('span', class_='prices-main-price')
        precio_text = precio.get_text(strip=True) if precio else '0'
        precio_cleaned = int(precio_text.replace('$', '').replace('.', ''))

        link = producto.find('a', class_='product-card-image-link')
        link_text = link.get('href') if link else 'Sin enlace'

        fila = { 'Supermercado': 'Jumbo',
                 'Marca': marca_text,
                 'Producto': nombre_text,
                 'Precio': precio_cleaned,
                 'Categoria': busqueda,
                 'Enlace': f'https://www.jumbo.cl{link_text}',
                 'Fecha': fecha,
                 'porMayor': ''
                 }
        
        tabla.loc[len(tabla)] = fila
        
    print(f'Jumbo - Scrap realizado {busqueda}')
        
    driver.quit()

# Otras funciones...

def scrap_santai(busqueda,tabla):
    
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    
    fecha = datetime.now().date()
    tiempo = randrange(5,10)
    driver = webdriver.Chrome(options=options)
    textobusqueda = transformar(busqueda)
    url = f'https://www.santaisabel.cl/busqueda?ft={textobusqueda}'
    
    driver.get(url)
    time.sleep(tiempo)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    productos = soup.find_all('div', class_='product-card')
    
    for producto in productos:
    
        marca = producto.find('a', class_='product-card-brand')
        marca_text = marca.get_text(strip=True).title() if marca else 'Sin marca'

        nombre = producto.find('a', class_='product-card-name')
        nombre_text = nombre.get_text(strip=True).title() if nombre else 'Sin nombre'

        precio = producto.find('span', class_='prices-main-price')
        precio_text = precio.get_text(strip=True) if precio else '0'
        precio_cleaned = int(precio_text.replace('$', '').replace('.', ''))

        link = producto.find('a', class_='product-card-image-link')
        link_text = link.get('href') if link else 'Sin enlace'

        fila = { 'Supermercado': 'Santa Isabel',
                 'Marca': marca_text,
                 'Producto': nombre_text,
                 'Precio': precio_cleaned,
                 'Categoria': busqueda,
                 'Enlace': f'https://www.santaisabel.cl{link_text}',
                 'Fecha': fecha,
                 'porMayor': ''
                 }
        
        tabla.loc[len(tabla)] = fila
        
    print(f'Santa Isabel - Scrap realizado {busqueda}')
    
    driver.quit()

def scrap_lider(busqueda,tabla):
    
    options = uc.ChromeOptions()
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    
    fecha = datetime.now().date()
    tiempo = randrange(12,24)
    driver = uc.Chrome(options=options)
    textobusqueda = transformar(busqueda)
    url = f'https://www.lider.cl/supermercado/search?query={textobusqueda}'
    
    driver.get(url)
    time.sleep(tiempo)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    productos = soup.find_all('li', class_='ais-Hits-item')
    
    for producto in productos:
    
        nombre = producto.find('h2', class_='product-description')
        
        if nombre:
            marca_span = nombre.find('span', style='font-weight: bold; color: rgb(0, 0, 0);')
            marca_text = marca_span.get_text(strip=True).title() if marca_span else 'Sin marca'
            
            nombre_span = marca_span.find_next_sibling('span') if marca_span else None
            nombre_text = nombre_span.get_text(strip=True).title() if nombre_span else 'Sin nombre'
        else:
            marca_text = 'Sin marca'
            nombre_text = 'Sin nombre'

        precio = producto.find('div', class_='product-card__sale-price')
        precio_text = precio.get_text(strip=True) if precio else '0'

        match = re.match(r'(\d+)\s*x\s*\$([\d.]+)', precio_text)
        if match:
            cantidad = int(match.group(1))
            precio_total = int(match.group(2).replace('.', ''))
            precio_cleaned = precio_total // cantidad
            por_mayor_text = precio_text
            
        else:
            precio_cleaned = int(precio_text.replace('$', '').replace('.', ''))
            por_mayor_text = ''

        link = producto.find('a', {'data-testid': 'product-card-nav-test-id'})
        link_text = link.get('href') if link else 'Sin enlace'

        fila = { 'Supermercado': 'Lider',
                 'Marca': marca_text,
                 'Producto': nombre_text,
                 'Precio': precio_cleaned,
                 'Categoria': busqueda,
                 'Enlace': f'https://www.lider.cl{link_text}',
                 'Fecha': fecha,
                 'porMayor': por_mayor_text
                 }
        
        tabla.loc[len(tabla)] = fila
        
        time.sleep(randrange(2,5))
        
    print(f'Lider - Scrap realizado {busqueda}')
        
    driver.quit()
    
def scrap_unimarc(busqueda,tabla):
    
    options = webdriver.ChromeOptions()
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    
    fecha = datetime.now().date()
    tiempo = randrange(5,10)
    driver = webdriver.Chrome(options=options)
    textobusqueda = busqueda.replace(' ', '-')
    url = f'https://www.unimarc.cl/search?q={textobusqueda}'
    
    driver.get(url)
    time.sleep(tiempo)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    contenedores = soup.find_all('div', class_='baseContainer_container__TSgMX ab__shelves abc__shelves baseContainer_justify-start___sjrG baseContainer_align-start__6PKCY baseContainer_absolute-default--topLeft__lN1In')
    
    for contenedor in contenedores:
    
        producto = contenedor.find('div', class_='baseContainer_container__TSgMX baseContainer_justify-start___sjrG baseContainer_align-start__6PKCY baseContainer_flex-direction--column__iiccg baseContainer_absolute-default--topLeft__lN1In')
    
        marca = producto.find('p', class_='Shelf_brandText__sGfsS')
        marca_text = marca.get_text(strip=True).title() if marca else 'Sin marca'

        nombre = producto.find('p', class_='Shelf_nameProduct__CXI5M')
        nombre_text = nombre.get_text(strip=True).title() if nombre else 'Sin nombre'

        precio = producto.find('p', class_='Text_text__cB7NM Text_text--left__1v2Xw Text_text--flex__F7yuI Text_text--medium__rIScp Text_text--lg__GZWsa Text_text--primary__OoK0C Text_text__cursor--auto__cMaN1 Text_text--none__zez2n')
        precio_text = precio.get_text(strip=True) if precio else '0'
        
        match = re.match(r'(\d+)\s*x\s*\$([\d.]+)', precio_text)
        if match:
            cantidad = int(match.group(1))
            precio_total = int(match.group(2).replace('.', ''))
            precio_cleaned = precio_total // cantidad
            por_mayor_text = precio_text
            
        else:
            precio_cleaned = int(precio_text.replace('$', '').replace('.', ''))
            por_mayor_text = ''

        link = producto.find('a', class_='Link_link___5dmQ')
        link_text = link.get('href') if link else 'Sin enlace'

        fila = { 'Supermercado': 'Unimarc',
                 'Marca': marca_text,
                 'Producto': nombre_text,
                 'Precio': precio_cleaned,
                 'Categoria': busqueda,
                 'Enlace': f'https://www.unimarc.cl{link_text}',
                 'Fecha': fecha,
                 'porMayor': por_mayor_text
                 }
        
        tabla.loc[len(tabla)] = fila
        
    print(f'Unimarc - Scrap realizado {busqueda}')
        
    driver.quit()

#### Obtención de los datos

In [2]:
import pandas as pd

# Quitar comentario si se necesita crear una nueva tabla
#df = pd.DataFrame(columns=['Supermercado','Marca','Producto','Precio','Categoria', 'Enlace', 'Fecha', 'porMayor'])

# Importar tabla de github
df = pd.read_csv("https://raw.githubusercontent.com/1bryanvalenzuela/Comparador-Supermercados-Chile/refs/heads/main/Listado-Supermercado.csv", index_col=["Id"])

# Revisar tabla
df

,Supermercado,Marca,Producto,Precio,Categoria,Enlace,Fecha,porMayor
Id,,,,,,,,
0,Lider,Soprole,"Leche Entera Natural, 200 Ml",470,Leche Entera,https://www.lider.cl/supermercado/product/sku/...,2024-10-01,NaN
1,Lider,Costa,"Galletas Frac Bi Capuccino, 110 G",483,Frac,https://www.lider.cl/supermercado/product/sku/...,2024-10-01,NaN
2,Lider,Costa,"Galleta Frac Clásica, 110 G",483,Frac,https://www.lider.cl/supermercado/product/sku/...,2024-10-01,NaN
3,Lider,Costa,"Galleta Frac Chocolate, 110 G",483,Frac,https://www.lider.cl/supermercado/product/sku/...,2024-10-01,NaN
4,Lider,Costa,"Galleta Frac Bi Frutilla, 110 G",483,Frac,https://www.lider.cl/supermercado/product/sku/...,2024-10-01,NaN
...,...,...,...,...,...,...,...,...
410,Unimarc,Soprole,Pack Leche Entera Natural Soprole 12 Un De 1 L,14440,Leche Entera,https://www.unimarc.cl/product/leche-entera-na...,2024-10-01,NaN
411,Santa Isabel,Nido,Leche Polvo Nido Entera 1.35 Kg,14490,Leche Entera,https://www.santaisabel.cl/leche-en-polvo-ente...,2024-10-01,NaN
412,Unimarc,Colun,"Pack Leche Entera Natural Colun, Sin Tapa 12 U...",15350,Leche Entera,https://www.unimarc.cl/product/leche-entera-na...,2024-10-01,NaN


In [7]:
# Terminos a buscar en los scraper:
ListaBusqueda = ["Tallarín", "Jurel", "Leche Entera", "Frac", "Porotos"]

# Uso de scrapers
for texto in ListaBusqueda:
    scrap_lider(texto,df)
    scrap_unimarc(texto,df)
    scrap_santai(texto,df)
    scrap_jumbo(texto,df)
    time.sleep(30)

Lider - Scrap realizado Tallarín
Unimarc - Scrap realizado Tallarín
Santa Isabel - Scrap realizado Tallarín
Jumbo - Scrap realizado Tallarín
Lider - Scrap realizado Jurel
Unimarc - Scrap realizado Jurel
Santa Isabel - Scrap realizado Jurel
Jumbo - Scrap realizado Jurel
Lider - Scrap realizado Leche Entera
Unimarc - Scrap realizado Leche Entera
Santa Isabel - Scrap realizado Leche Entera
Jumbo - Scrap realizado Leche Entera
Lider - Scrap realizado Frac
Unimarc - Scrap realizado Frac
Santa Isabel - Scrap realizado Frac
Jumbo - Scrap realizado Frac
Lider - Scrap realizado Porotos
Unimarc - Scrap realizado Porotos
Santa Isabel - Scrap realizado Porotos
Jumbo - Scrap realizado Porotos


In [8]:
df

,Supermercado,Marca,Producto,Precio,Categoria,Enlace,Fecha,porMayor
Id,,,,,,,,
0,Lider,Soprole,"Leche Entera Natural, 200 Ml",470,Leche Entera,https://www.lider.cl/supermercado/product/sku/...,2024-10-01,NaN
1,Lider,Costa,"Galletas Frac Bi Capuccino, 110 G",483,Frac,https://www.lider.cl/supermercado/product/sku/...,2024-10-01,NaN
2,Lider,Costa,"Galleta Frac Clásica, 110 G",483,Frac,https://www.lider.cl/supermercado/product/sku/...,2024-10-01,NaN
3,Lider,Costa,"Galleta Frac Chocolate, 110 G",483,Frac,https://www.lider.cl/supermercado/product/sku/...,2024-10-01,NaN
4,Lider,Costa,"Galleta Frac Bi Frutilla, 110 G",483,Frac,https://www.lider.cl/supermercado/product/sku/...,2024-10-01,NaN
...,...,...,...,...,...,...,...,...
840,Jumbo,Hitecsa,Porotos Granados Con Mazamorra Greda,4590,Porotos,https://www.jumbo.cl/porotos-granados-con-maza...,2024-10-06,
841,Jumbo,Ilko,Rebanador De Porotos Ilko Basic,7290,Porotos,https://www.jumbo.cl/rebanador-de-porotos-ilko...,2024-10-06,
842,Jumbo,Iansa Agro,Porotos Rojos Listos 390 G,1389,Porotos,https://www.jumbo.cl/poroto-rojo-iansa/p,2024-10-06,


#### Limpieza y exportación

In [13]:
# Quitar resultados con valor a 0 (agotados o no disponibles en la tienda)
df2 = df[df['Precio'] != 0]

# Ordenar productos por precio de forma ascendente, reiniciar index y nombrarlo como "Id", para que los datos sean más fáciles de ordenar y leer en Power BI.
df3 = df2.sort_values(by=["Precio"], ascending=True).reset_index(drop=True)
df3.index.name = "Id"

# Tabla final
df3

,Supermercado,Marca,Producto,Precio,Categoria,Enlace,Fecha,porMayor
Id,,,,,,,,
0,Lider,Soprole,"Leche Entera Natural, 200 Ml",470,Leche Entera,https://www.lider.cl/supermercado/product/sku/...,2024-10-01,NaN
1,Lider,Soprole,"Leche Entera Natural, 200 Ml",470,Leche Entera,https://www.lider.cl/supermercado/product/sku/...,2024-10-06,
2,Lider,Costa,"Galleta Frac Clásica, 110 G",483,Frac,https://www.lider.cl/supermercado/product/sku/...,2024-10-01,NaN
3,Lider,Costa,"Galleta Frac Chocolate, 110 G",483,Frac,https://www.lider.cl/supermercado/product/sku/...,2024-10-01,NaN
4,Lider,Costa,"Galleta Frac Bi Frutilla, 110 G",483,Frac,https://www.lider.cl/supermercado/product/sku/...,2024-10-01,NaN
...,...,...,...,...,...,...,...,...
825,Unimarc,Colun,"Pack Leche Entera Natural Colun, Sin Tapa 12 U...",15350,Leche Entera,https://www.unimarc.cl/product/leche-entera-na...,2024-10-06,
826,Unimarc,Colun,"Pack Leche Entera Natural Colun, Sin Tapa 12 U...",15350,Leche Entera,https://www.unimarc.cl/product/leche-entera-na...,2024-10-01,NaN
827,Jumbo,Colun,Pack 12 Un. Leche Colun Entera Sin Lactosa 1 L,16668,Leche Entera,https://www.jumbo.cl/pack-leche-entera-sin-lac...,2024-10-01,NaN


In [14]:
import requests
import base64
from io import StringIO

# Exportación del CSV final

# Quitar comentario para guardar localmente (configurar ruta)
#df3.to_csv('C:/Users/Bryan/Documents/Portafolio/Supermercados/Listado-Supermercado.csv')

# Detalles del archivo y del repositorio
repo = "1bryanvalenzuela/Comparador-Supermercados-Chile"
csv = "Listado-Supermercado.csv"
branch = "main"
token = ""

# Convertir el DataFrame a un CSV en memoria usando StringIO
csv_buffer = StringIO()
df3.to_csv(csv_buffer, index=True, index_label="Id")

# Obtener el contenido del archivo en formato string y codificarlo en base64
contenido_csv = csv_buffer.getvalue()
contenido_encoded = base64.b64encode(contenido_csv.encode()).decode()

# URL de la API de GitHub para obtener el archivo actual
url_obtener = f"https://api.github.com/repos/{repo}/contents/{csv}"

# Realizar la solicitud para obtener el archivo existente
respuesta_obtener = requests.get(url_obtener, headers={"Authorization": f"token {token}"})

# Si el archivo existe, obtenemos el SHA
if respuesta_obtener.status_code == 200:
    sha = respuesta_obtener.json()["sha"]
else:
    sha = None

# URL de la API de GitHub para subir archivos
url = f"https://api.github.com/repos/{repo}/contents/{csv}"

# Crear el payload para la solicitud
data = {
    "message": "CSV Añadido",
    "content": contenido_encoded,
    "branch": branch
}

if sha:
    data["sha"] = sha

# Realizar la solicitud a la API de GitHub
respuesta = requests.put(url, json=data, headers={"Authorization": f"token {token}"})

# Mostrar resultado de la subida
if respuesta.status_code in [200, 201]:
    print("Archivo subido correctamente.")
else:
    print(f"Error al subir el archivo: {respuesta.status_code}")

Archivo subido correctamente.
